In [104]:
#The coalesce method reduces the number of partitions in a DataFrame.
df = spark.read.format("csv").option("header" ,"true").option("inferSchema" , "true")\
.load("C:/Users/dewair/Spark-The-Definitive-Guide-master/data/retail-data/all/*.csv").coalesce(5)
df.cache()
df.createOrReplaceTempView("dftable")

In [105]:
df.count() # we use it here as action 

541909

In [106]:
spark.sql("SELECT count(*) FROM dftable").show() # in sql  count(1) will count the first column

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [107]:
from pyspark.sql.functions import count
df.select (count (df.StockCode)).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [108]:
from pyspark.sql.functions import countDistinct
df.select (countDistinct (df.StockCode)).show()
spark.sql("select count (distinct (StockCode)) from dftable").show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [109]:
#select the first and last row in DF , will be based on the df , not on values within df
from pyspark.sql.functions import first, last
df.select (first(df.StockCode), last(df.StockCode)).show()
spark.sql("select first(StockCode) ,  last(StockCode) from dftable").show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [110]:
# extract the min and max value 
from pyspark.sql.functions import min, max
df.select (min(df.Quantity), max(df.Quantity)).show()
spark.sql("select min(Quantity) ,  max(Quantity) from dftable").show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [111]:
# extract the sum
from pyspark.sql.functions import sum
df.select (sum(df.Quantity)).show()
spark.sql("select sum(Quantity) from dftable").show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [112]:
# extract the sumdistinct
from pyspark.sql.functions import sumDistinct
df.select (sumDistinct(df.Quantity)).show()
spark.sql("select sum(distinct (Quantity )) from dftable").show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [113]:
#mean ,avg
from pyspark.sql.functions import sum , count , avg , expr
df.select (sum(df.Quantity).alias("sum_qut") ,count(df.Quantity).alias("count_qut"),\
          avg(df.Quantity).alias("avg_qut") , expr ("mean(Quantity)").alias("mean_qut"))\
.selectExpr("sum_qut/count_qut" , "avg_qut" ,"mean_qut" ).show(2)

+---------------------+----------------+----------------+
|(sum_qut / count_qut)|         avg_qut|        mean_qut|
+---------------------+----------------+----------------+
|     9.55224954743324|9.55224954743324|9.55224954743324|
+---------------------+----------------+----------------+



## group by 

In [114]:
# count the occurences of each invoice 
from pyspark.sql.functions import count 
df.groupBy(df.InvoiceNo).agg(count(df.Quantity).alias("quan")\
                         , expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



In [115]:
spark.sql("select avg(Quantity) ,  InvoiceNo from dftable group by InvoiceNo ").show(5)

+------------------+---------+
|     avg(Quantity)|InvoiceNo|
+------------------+---------+
|               1.5|   536596|
|33.142857142857146|   536938|
|              31.0|   537252|
|              8.15|   537691|
|              30.0|   538041|
+------------------+---------+
only showing top 5 rows



In [116]:
## window functions
from pyspark.sql.functions import to_date 
dfwithdate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm")) #d must be one didgit 
dfwithdate.createOrReplaceTempView("dfwithdate")

In [117]:
dfwithdate.show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
only showing top 2 rows



In [118]:
# the first part to a window function is to create window specifications
# partitionBy : describe how we will be breaking up our group
# orderBY :  determine the ordering within a given partition 
# rowsBetween : atates which rows will be included based on current row

In [119]:
# here i'm defining window specification
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window.partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [120]:
# now it's time to create aggregate functions to run over window specification
# i want to calculate the maximum purchase quatity over time
from pyspark.sql.functions import max , col
maxpurchase = max(col("Quantity")).over(windowSpec)# this will return a column , we can use it with select

In [121]:
# we will create dense_rank :determine which date had the maximum purchase quantity
from pyspark.sql.functions import rank, dense_rank
purchasedense_rank = dense_rank().over(windowSpec)
purchaserank = rank().over(windowSpec)
# these two functions will return two columns , we can use them in select 

In [122]:
dfwithdate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaserank.alias("quantityRank"),
    purchasedense_rank.alias("quantityDenseRank"),
    maxpurchase.alias("maxPurchaseQuantity")).show()


+----------+----+--------+------------+-----------------+-------------------+
|CustomerId|date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----+--------+------------+-----------------+-------------------+
|     12346|null|   74215|           1|                1|              74215|
|     12346|null|  -74215|           2|                2|              74215|
|     12347|null|     240|           1|                1|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      36|           2|                2|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|                240|
|     12347|null|      24|           5|                3|       

In [95]:
dfnonull = dfwithdate.drop()

In [103]:
# we will stop here in page 128 , in grouping set 